## controlnet

* https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/controlnet.ipynb#scrollTo=wsv55Py8onJx

In [1]:
import os, glob
import random, math
import numpy as np
import pandas as pd
import torch
styles = pd.read_csv('stable_diffusion_artist_styles.csv')

In [2]:
from diffusers.utils import load_image

def conditioning(image):
    import cv2
    from PIL import Image

    image = np.array(image)

    low_threshold = 50
    high_threshold = 200

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    canny_image = Image.fromarray(image)
    return canny_image

/storage/local/stablediff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16, safety_checker=None)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)
from diffusers import UniPCMultistepScheduler

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_xformers_memory_efficient_attention()
pipe.safety_checker = lambda images, clip_input: (images, False)

/storage/local/stablediff/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:


In [4]:
def controlnet_prompt(prompt, canny_img, n=1, style=None, path='.'):
    if style != None:
        prompt += ' by %s'%style
    #prompt += ", best quality, extremely detailed, studio photograph, cinematic"
    for c in range(n):
        random_seed = np.random.randint(1000)
        generator = torch.Generator(device="cpu").manual_seed(random_seed)
        output = pipe(
            prompt,
            canny_img,
            negative_prompt="disfigured, monochrome, lowres, bad anatomy, worst quality, low quality" * len(prompt),
            generator=generator,
            num_inference_steps=20,
        )
        image = output.images[0]
        if not os.path.exists(path):
            os.makedirs(path)        
        i=1
        imgfile = os.path.join(path,prompt[:90]+'_%s.png' %i)
        while os.path.exists(imgfile):
            i+=1
            imgfile = os.path.join(path,prompt[:90]+'_%s.png' %i)
        image.save(imgfile,'png')           
    return image

In [ ]:
#image = load_image('superman-pose2.jpg')
image = load_image('uvdl.jpg')
canny_image = conditioning(image)
canny_image

In [ ]:
controlnet_prompt("",
                  canny_image,n=2,style='',path='controlnet')

In [9]:
def make_montage(cond_img, names, style, path, other=''):
    """montage from conditional image"""

    canny_image = conditioning(cond_img)
    for name in names:
        controlnet_prompt(name+' '+other,cond_img,n=1,style=style,path=path)
    return

In [ ]:
pres=['lyndon johnson','dwight eisenhower','harry truman','woodrow wilson','teddy roosevelt','franklin roosevelt',
       'john f kennedy','richard nixon','gerald ford','jimmy carter','joe biden',
      'donald trump','george w bush','george hw bush','bill clinton','barack obama'
      ]
actors = ['tom cruise','brad pitt','chris pine','arnold schwarzenegger',
          'sylvester stallone','chris hemsworth','chris pratt','henry cavill'
          ]
make_montage(canny_image, actors, 'robert bateman', 'pose', other='superhero')

In [ ]:
def make_gif(path, outfile):
    """make gif from same condition image"""

    import glob
    from PIL import Image
    files = glob.glob(path+'/*.png')
    #print (files)
    img = Image.open(files[0])    
    #size = img.size
    images=[]
    images = [Image.open(f) for f in files]
    images[0].save(outfile,
               save_all=True, append_images=images[1:], 
               optimize=False, duration=400, loop=0)
    return

make_gif('superman','superman.gif')

In [46]:
def tile_images(image_paths, outfile, grid=False, tile_width=300):
    """Make tiled image"""

    from PIL import Image, ImageDraw
    images = [Image.open(path) for path in image_paths]
      
    ratio = images[0].height / images[0].width
    tile_height = int( tile_width * ratio )
    num_rows = int(math.sqrt(len(image_paths)))
    # Calculate number of cols
    num_columns = (len(images) + num_rows - 1) // num_rows

    tiled_width = num_columns * tile_width
    tiled_height = num_rows * tile_height
    tiled_image = Image.new("RGB", (tiled_width, tiled_height))

    for idx, image in enumerate(images):      
        row = idx // num_columns
        col = idx % num_columns
        x_offset = col * tile_width
        y_offset = row * tile_height
        tiled_image.paste(image.resize((tile_width, tile_height)), (x_offset, y_offset))
    if grid == True:
        draw = ImageDraw.Draw(tiled_image)
        # Draw borders around each tile
        for row in range(num_rows):
            for col in range(num_columns):
                x1 = col * tile_width
                y1 = row * tile_height
                x2 = x1 + tile_width
                y2 = y1 + tile_height
                draw.rectangle([x1, y1, x2, y2], outline=(0, 0, 0), width=3)  

    tiled_image.save(outfile)
    return tiled_image

In [ ]:
files = glob.glob('mona/*.png')
x = random.sample(files, 3)
tile_images(x, 'tiled.png', grid=True)